### Finding Cut-off Temperatures for Controlling and decision making


This short script looks for the temperature at which the heat pump is to be turned off in every cycle, depending on the nominal power. Therefore we optimize one cycle for every nominal power between 3 and 4kW, with the condition, that the starting temperature must be equal to the ending temperature. Note that the higher powers don't manage to fulfill this constraint and ouput "None".

In [1]:

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt

### COP and Heat fit 

### Reading in the data for the fits

In [2]:
import numpy as np

# Degree of polynomial for HEAT_DATA fitting
HEAT_POLY_DEGREE = 3

# Data containers
data_dict = {}  # { (DIA, FLUID): {"COP_DATA": {...}, "HEAT_DATA": {...}} }
heat_poly_coeffs_dict = {}  # { (DIA, FLUID): coeffs }
optimization_results = {}  # { (DIA, FLUID): optimization_result }

# --- STEP 1: Read and parse the text file ---
with open("heat_pump_results.txt", "r") as f:
    current_key = None
    current_section = None
    cop_data = {}
    heat_data = {}

    for line in f:
        line = line.strip()

        if not line:
            continue

        if line.startswith("DIA="):
            if current_key and heat_data:
                data_dict[current_key] = {
                    "COP_DATA": cop_data,
                    "HEAT_DATA": heat_data
                }
                cop_data = {}
                heat_data = {}

            dia = line.split(",")[0].split("=")[1]
            fluid = line.split(",")[1].split("=")[1]
            current_key = (dia, fluid)

        elif line == "COP_DATA:":
            current_section = "COP"
        elif line == "HEAT_DATA:":
            current_section = "HEAT"
        elif line == "END_ENTRY":
            if current_key:
                data_dict[current_key] = {
                    "COP_DATA": cop_data,
                    "HEAT_DATA": heat_data
                }
                cop_data = {}
                heat_data = {}
                current_key = None
                current_section = None
        else:
            try:
                t, v = map(float, line.split(","))
                if current_section == "COP":
                    cop_data[t] = v
                elif current_section == "HEAT":
                    heat_data[t] = v
            except ValueError:
                continue

# --- STEP 2: Define fit and optimization functions ---

def generate_heat_fit(heat_dict):
    if not heat_dict:
        raise ValueError("No heat data to fit.")
    temps, heat_vals = zip(*sorted(heat_dict.items()))
    return np.polyfit(temps, heat_vals, HEAT_POLY_DEGREE)

def get_heat_from_fit(coeffs, T):
    return float(np.polyval(coeffs, T))

def print_heat_fit_equation(coeffs, display=True):
    terms = [f"{coef:.6g} * T^{HEAT_POLY_DEGREE - i}" 
             for i, coef in enumerate(coeffs)]
    equation = " + ".join(terms)
    if display:
        print("Heat(T) ≈", equation)
    return list(coeffs)


# Dummy optimization function to simulate usage
def optimize_with_fit(coeffs, key):
    # Replace this with your real optimization logic
    # For example: minimize energy usage, simulate performance, etc.
    predicted_at_50 = get_heat_from_fit(coeffs, 50)
    return {"PredictedHeatAt50C": predicted_at_50}


In [3]:

# --- STEP 3: Loop through all entries and run fit + optimization ---

for key, value in data_dict.items():
    dia, fluid = key
    heat_dict = value["HEAT_DATA"]

    try:
        coeffs = generate_heat_fit(heat_dict)
        coeff_list = list(coeffs)  # Clean usable list
        heat_poly_coeffs_dict[key] = coeff_list

        print(f"\n--- {fluid} ({dia} mm) ---")
        print_heat_fit_equation(coeff_list)  # Still prints readable version
        print("Stored Coefficients:", coeff_list)

        result = optimize_with_fit(coeff_list, key)
        optimization_results[key] = {
            "coeffs": coeff_list,
            "optimization": result
        }

        print("Optimization Result:", result)

    except Exception as e:
        print(f"Failed for {key}: {e}")


# --- Optional: Summary ---
print("\n--- All Results Summary ---")
for key, result in optimization_results.items():
    print(f"{key}: {result}")
    



--- Isobutane (35 mm) ---
Heat(T) ≈ 3.14093e-07 * T^3 + -0.000108094 * T^2 + -0.0394197 * T^1 + 7.9088 * T^0
Stored Coefficients: [3.140927258163184e-07, -0.00010809446667183591, -0.03941969824764947, 7.908802539048254]
Optimization Result: {'PredictedHeatAt50C': 5.70684305071323}

--- Butane (35 mm) ---
Heat(T) ≈ 4.45705e-07 * T^3 + -0.000126931 * T^2 + -0.0261079 * T^1 + 5.89016 * T^0
Stored Coefficients: [4.4570495779968916e-07, -0.00012693106022569755, -0.026107876372625696, 5.890159814653347]
Optimization Result: {'PredictedHeatAt50C': 4.32315146518278}

--- Isobutene (35 mm) ---
Heat(T) ≈ 3.30586e-07 * T^3 + -0.000118888 * T^2 + -0.0315025 * T^1 + 7.08577 * T^0
Stored Coefficients: [3.3058602249160847e-07, -0.00011888751820712992, -0.03150253799267094, 7.085771459000568]
Optimization Result: {'PredictedHeatAt50C': 5.2547490166606465}

--- DimethylEther (35 mm) ---
Heat(T) ≈ 2.39387e-07 * T^3 + -0.000170109 * T^2 + -0.0464024 * T^1 + 12.2433 * T^0
Stored Coefficients: [2.39387337

In [4]:
coeffs = heat_poly_coeffs_dict[("50", "Isobutane")]


In [5]:
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

temperature_results = {}

all_keys = list(data_dict.keys())
all_diams = sorted(set(key[0] for key in all_keys))
all_fluids = sorted(set(key[1] for key in all_keys))

for dia, fluid in product(all_diams, all_fluids):
    print(f"\n🔄 Processing DIA={dia} mm, FLUID={fluid}")
    key = (dia, fluid)

    if key not in data_dict:
        continue

    heat_coeffs = heat_poly_coeffs_dict.get(key, None)
    if heat_coeffs is None or len(heat_coeffs) != 4:
        continue

    try:
        dt = 1
        cook_time = 7 * 60
        cool_time = 5 * 60
        total_time = cook_time + cool_time
        n_steps = total_time + 1

        T_env = 20
        T_pasta_0 = 7
        R_env = 30 / 1000
        R_pasta = 15 / 1000
        c_l = 4.18 * 1000
        c_pasta = 3.5 * 1000

        V_water = (47 - 0.4) * (27 - 0.4) * (20 - 0.2) / 1e6
        m_water = V_water * 1000
        m_pasta = 2

        C_water = m_water * c_l
        C_pasta = m_pasta * c_pasta

        model = gp.Model("Thermal_Cooking_Optimized")
        model.setParam("OutputFlag", 0)
        model.setParam("NonConvex", 2)

        z = model.addVars(n_steps, vtype=GRB.BINARY, name="z")
        u = model.addVars(n_steps, vtype=GRB.BINARY, name="u")
        Q = model.addVars(n_steps, lb=0, name="Q")
        T_water = model.addVars(n_steps, lb=0, name="T_water")
        T_pasta = model.addVars(n_steps, lb=0, name="T_pasta")

        model.addConstr(T_pasta[0] == T_pasta_0)

        # Detect when heat pump turns on
        #model.addConstr(z[0] == u[0])
        #for t in range(1, n_steps):
        #    model.addConstr(u[t] >= z[t] - z[t - 1])
        #model.addConstr(gp.quicksum(u[t] for t in range(n_steps)) <= 2)

        for t in range(n_steps):
            T = T_water[t]
            T_sq = model.addVar(lb=0, name=f"T_sq_{t}")
            T_cu = model.addVar(lb=0, name=f"T_cu_{t}")
            model.addConstr(T_sq == T * T)
            model.addConstr(T_cu == T_sq * T)
            model.addConstr(Q[t] == (heat_coeffs[0] * T_cu +
                                     heat_coeffs[1] * T_sq +
                                     heat_coeffs[2] * T +
                                     heat_coeffs[3]) * z[t])

        for t in range(1, n_steps):
            if t <= cook_time:
                model.addConstr(
                    T_water[t] == T_water[t - 1] + dt * (Q[t - 1] * 1000 - (T_water[t - 1] - T_env) / R_env - (T_water[t - 1] - T_pasta[t - 1]) / R_pasta) / C_water
                )
                model.addConstr(
                    T_pasta[t] == T_pasta[t - 1] + dt * ((T_water[t - 1] - T_pasta[t - 1]) / R_pasta) / C_pasta
                )
            else:
                model.addConstr(
                    T_water[t] == T_water[t - 1] + dt * (Q[t - 1] * 1000 - (T_water[t - 1] - T_env) / R_env) / C_water
                )
                model.addConstr(T_pasta[t] == T_pasta[t - 1])

        for t in range(n_steps):
            model.addConstr(T_water[t] >= 85)
            model.addConstr(T_water[t] <= 93)

        model.addConstr(T_pasta[cook_time] >= 85)
        model.addConstr(T_pasta[cook_time] <= 85.5)

        total_energy = gp.quicksum(Q[t] * dt / 3600 for t in range(n_steps))
        model.setObjective(total_energy, GRB.MINIMIZE)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            T_water_vals = np.array([T_water[i].X for i in range(n_steps)])
            T_pasta_vals = np.array([T_pasta[i].X for i in range(n_steps)])
            Q_vals = np.array([Q[i].X for i in range(n_steps)])
            z_vals = np.array([z[i].X for i in range(n_steps)])

            t_minutes = np.arange(n_steps) / 60
            a, b, c, d = heat_coeffs
            Q_actual_vals = a * T_water_vals**3 + b * T_water_vals**2 + c * T_water_vals + d

            fig, ax1 = plt.subplots(figsize=(12, 6))
            ax1.plot(t_minutes, T_water_vals, 'b-', label='Water Temp (°C)')
            ax1.plot(t_minutes[:cook_time], T_pasta_vals[:cook_time], 'g--', label='Pasta Temp (°C)')
            ax1.hlines([85, 93], 0, t_minutes[-1], colors='b', linestyles='dotted', linewidth=1)
            ax1.hlines(85, 0, t_minutes[-1], colors='g', linestyles='dotted', linewidth=1)
            ax1.set_xlabel("Time (min)")
            ax1.set_ylabel("Temperature (°C)", color='k')
            ax1.grid(True)

            ax2 = ax1.twinx()
            ax2.plot(t_minutes, Q_actual_vals * z_vals, 'm-', linewidth=2, label='Heat Transferred (kW)')
            ax2.set_ylabel("Heat Transferred (kW)", color='m')

            ax3 = ax1.twinx()
            ax3.spines["right"].set_position(("axes", 1.1))
            ax3.plot(t_minutes, z_vals, 'k:', linewidth=1.5, label='Heat Pump On (z)')
            ax3.set_ylabel("Heat Pump On/Off", color='k')
            ax3.set_yticks([0, 1])

            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            lines3, labels3 = ax3.get_legend_handles_labels()
            ax1.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='lower right')

            plt.title(f"Thermal Cooking: DIA={dia}, FLUID={fluid}")
            plt.tight_layout()
            plt.show()

            temperature_results[key] = {
                "T_water_start": float(T_water_vals[0]),
                "T_water_max": float(T_water_vals.max()),
                "T_pasta_max": float(T_pasta_vals.max())
            }

    except Exception as e:
        print(f"❌ Error processing {key}: {e}")
        continue



🔄 Processing DIA=35 mm, FLUID=Butane
Set parameter Username
Set parameter LicenseID to value 2653942


Academic license - for non-commercial use only - expires 2026-04-17

Interrupt request received

🔄 Processing DIA=35 mm, FLUID=DimethylEther


In [6]:
#print (temperature_results)
# Print pasta temperature at the end of cooking
for key, temps in temperature_results.items():
    print(f"Final Pasta Temperature for DIA={key[0]} mm, FLUID={key[1]}: {temps['T_pasta_max']:.2f} °C")

NameError: name 'temperature_results' is not defined

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

# Nested dictionary to store results
temperature_results = {}

# Step 1: Extract unique refrigerants and diameters from keys
all_keys = list(data_dict.keys())
all_diams = sorted(set(key[0] for key in all_keys))
all_fluids = sorted(set(key[1] for key in all_keys))

# Step 2: Iterate through all combinations
print("Iterating over all (DIA, FLUID) combinations:\n")

for dia, fluid in product(all_diams, all_fluids):
    print(f"\n🔄 Processing DIA={dia} mm, FLUID={fluid}")
    key = (dia, fluid)

    if key not in data_dict:
        print(f"❌ No data for {key}")
        continue

    heat_coeffs = heat_poly_coeffs_dict.get(key, None)
    if heat_coeffs is None or len(heat_coeffs) != 4:
        print(f"❌ Invalid or missing heat coefficients for {key}")
        continue

    try:
        # --- Time and Cooking Parameters ---
        dt = 1  # seconds
        cycle_time = 12 * 60  # seconds
        cook_time = 7 * 60
        cool_time = 5 * 60
        n_cycles = 60
        total_time = cycle_time * n_cycles
        n_steps = total_time + 1

        # --- Environment and Physical Properties ---
        T_env = 20
        T_pasta_0 = 7
        R_env = 30 / 1000
        R_pasta = 15 / 1000
        c_l = 4.18 * 1000
        c_pasta = 3.5 * 1000

        V_water = (47 - 0.4) * (27 - 0.4) * (20 - 0.2) / 1e6
        m_water = V_water * 1000
        m_pasta = 2

        C_water = m_water * c_l
        C_pasta = m_pasta * c_pasta

        # --- Gurobi Model ---
        model = gp.Model("Thermal_Cooking_Optimized")
        model.setParam("OutputFlag", 0)
        model.setParam("NonConvex", 2)

        z = model.addVars(n_steps, vtype=GRB.BINARY, name="z")
        on = model.addVars(n_steps, vtype=GRB.BINARY, name="on")
        Q = model.addVars(n_steps, lb=0, name="Q")
        T_water = model.addVars(n_steps, lb=0, name="T_water")
        T_pasta = model.addVars(n_steps, lb=0, name="T_pasta")

        model.addConstr(T_pasta[0] == T_pasta_0)
        model.addConstr(T_water[0] >= 85)
        model.addConstr(T_water[0] <= 93)

        for t in range(1, n_steps):
            model.addConstr(on[t] >= z[t] - z[t - 1])

        for cycle in range(n_cycles):
            start = cycle * cycle_time
            end = min((cycle + 1) * cycle_time, n_steps)
            model.addConstr(gp.quicksum(on[t] for t in range(start + 1, end)) <= 1)

        for t in range(n_steps):
            T = T_water[t]
            T_sq = model.addVar(lb=0, name=f"T_sq_{t}")
            T_cu = model.addVar(lb=0, name=f"T_cu_{t}")
            model.addConstr(T_sq == T * T)
            model.addConstr(T_cu == T_sq * T)
            model.addConstr(Q[t] == (heat_coeffs[0] * T_cu + heat_coeffs[1] * T_sq + heat_coeffs[2] * T + heat_coeffs[3]) * z[t])

        for t in range(1, n_steps):
            in_cycle_time = t % cycle_time
            if in_cycle_time <= cook_time:
                model.addConstr(T_water[t] == T_water[t - 1] + dt * (Q[t - 1] * 1000 - (T_water[t - 1] - T_env) / R_env - (T_water[t - 1] - T_pasta[t - 1]) / R_pasta) / C_water)
                model.addConstr(T_pasta[t] == T_pasta[t - 1] + dt * ((T_water[t - 1] - T_pasta[t - 1]) / R_pasta) / C_pasta)
            else:
                model.addConstr(T_water[t] == T_water[t - 1] + dt * (Q[t - 1] * 1000 - (T_water[t - 1] - T_env) / R_env) / C_water)
                model.addConstr(T_pasta[t] == T_pasta[t - 1])

        for t in range(n_steps):
            model.addConstr(T_water[t] >= 85)
            model.addConstr(T_water[t] <= 93)

        for cycle in range(n_cycles):
            t_check = (cycle + 1) * cycle_time - cool_time  # end of cook phase
            model.addConstr(T_pasta[t_check] >= 85)
            model.addConstr(T_pasta[t_check] <= 85.5)

        total_energy = gp.quicksum(Q[t] * dt / 3600 for t in range(n_steps))
        model.setObjective(total_energy, GRB.MINIMIZE)
        model.optimize()

        if model.status == GRB.OPTIMAL:
            T_water_vals = np.array([T_water[i].X for i in range(n_steps)])
            T_pasta_vals = np.array([T_pasta[i].X for i in range(n_steps)])
            Q_vals = np.array([Q[i].X for i in range(n_steps)])
            z_vals = np.array([z[i].X for i in range(n_steps)])

            t_minutes = np.arange(n_steps) / 60
            a, b, c, d = heat_coeffs
            Q_actual_vals = a * T_water_vals**3 + b * T_water_vals**2 + c * T_water_vals + d

            fig, ax1 = plt.subplots(figsize=(12, 6))
            ax1.plot(t_minutes, T_water_vals, 'b-', label='Water Temp (\u00b0C)')
            ax1.plot(t_minutes, T_pasta_vals, 'g--', label='Pasta Temp (\u00b0C)')
            ax1.hlines([85, 93], 0, t_minutes[-1], colors='b', linestyles='dotted', linewidth=1)
            ax1.hlines(85, 0, t_minutes[-1], colors='g', linestyles='dotted', linewidth=1)
            ax1.set_xlabel("Time (min)")
            ax1.set_ylabel("Temperature (\u00b0C)", color='k')
            ax1.grid(True)

            ax2 = ax1.twinx()
            ax2.plot(t_minutes, Q_actual_vals * z_vals, 'm-', linewidth=2, label='Heat Transferred (kW)')
            ax2.set_ylabel("Heat Transferred (kW)", color='m')

            ax3 = ax1.twinx()
            ax3.spines["right"].set_position(("axes", 1.1))
            ax3.plot(t_minutes, z_vals, 'k:', linewidth=1.5, label='Heat Pump On (z)')
            ax3.set_ylabel("Heat Pump On/Off", color='k')
            ax3.set_yticks([0, 1])

            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            lines3, labels3 = ax3.get_legend_handles_labels()
            ax1.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='lower right')

            plt.title(f"Thermal Cooking: DIA={dia}, FLUID={fluid}")
            plt.tight_layout()
            plt.show()

            temperature_results[key] = {
                "T_water_start": float(T_water_vals[0]),
                "T_water_max": float(T_water_vals.max()),
                "T_pasta_max": float(T_pasta_vals.max())
            }

            print(f"✅ Success for {key} | T_pasta_max = {T_pasta_vals.max():.2f} °C")

        else:
            print(f"⚠️ Gurobi returned status {model.status} for {key}")

    except Exception as e:
        print(f"❌ Error processing {key}: {e}")
        continue

Iterating over all (DIA, FLUID) combinations:


🔄 Processing DIA=35 mm, FLUID=Butane
Set parameter Username
Set parameter LicenseID to value 2653942


Academic license - for non-commercial use only - expires 2026-04-17

Interrupt request received
